In [34]:
import os
import pandas as pd
import numpy as np
from glob import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import shutil
from tqdm import tqdm
import zipfile

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2

import timm
import wandb

import warnings
warnings.filterwarnings("ignore")

In [35]:
PROJECT_DIR = os.getcwd()
DATA_DIR = os.path.join(PROJECT_DIR, "data")

# Train Validation Split

- 트레인 데이터와 검증데이터를 홀드 아웃 기법을 통하여 데이터를 분할한다.
- 트레인 데이터와 검증데이터의 비율은 각각 0.8 : 0.2로 분할한다.

In [6]:
def SplitDataset(img_dir:str, val_size:float=0.1, seed=42):
    fake_images = glob(f'{img_dir}/fake_images/*')
    real_images = glob(f'{img_dir}/real_images/*')
    labels = [1] * len(fake_images) + [0] * len(real_images)

    X_train, X_val, y_train, y_val = train_test_split(fake_images + real_images, labels, test_size=val_size, random_state=seed, shuffle=True)

    return X_train, X_val, y_train, y_val

In [7]:
X_train, X_val, y_train, y_val = SplitDataset("./data/train", val_size=0.2, seed=42)
test_images = glob("./data/test/images/*")

In [8]:
train = pd.DataFrame({"path" : X_train,
              "label" : y_train})
val = pd.DataFrame({"path" : X_val,
                    "label" : y_val})
test = pd.DataFrame({"path" : test_images})

train["path"] = train["path"].str.replace("\\", "/")
val["path"] = val["path"].str.replace("\\", "/")
test["path"] = test["path"].str.replace("\\", "/")
train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)
test.to_csv("test.csv", index=False)

In [9]:
X_train = pd.read_csv("./train.csv")
X_val = pd.read_csv("./val.csv")

In [10]:
X_train["label"].value_counts()

label
0    12193
1    12093
Name: count, dtype: int64

In [11]:
X_val["label"].value_counts()

label
1    3067
0    3005
Name: count, dtype: int64

# 데이터 이동

- 검증데이터로 분할된 데이터들을 추가 폴더 생성을 통하여 검증데이터만 따로 새로운 폴더를 만들어서 이동시킨다.

In [14]:
# 대상 폴더 생성
os.makedirs('./data/val/fake_images', exist_ok=True)
os.makedirs('./data/val/real_images', exist_ok=True)

# 파일 이동
for index, row in X_val.iterrows():
    src_path = row['path']
    filename = os.path.basename(src_path)
    label = row['label']
    
    if label == 1:
        dst_path = f'./data/val/fake_images/{filename}'
    else:
        dst_path = f'./data/val/real_images/{filename}'
    
    shutil.move(src_path, dst_path)

# 레이블 업데이트
X_val['path'] = X_val.apply(lambda row: f'./data/val/fake_images/{os.path.basename(row["path"])}' if row['label'] == 1 else f'./data/val/real_images/{os.path.basename(row["path"])}', axis=1)


In [16]:
X_val.to_csv("./val.csv", index=False)

# 표준편차, 정규화

- 이 데이터의 표준편차와 정규화를 알기 위하여 train 데이터들의 평균과 표준편차를 확인

In [20]:
def get_mean_std(data_dir):

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    dataset = datasets.ImageFolder(os.path.join(f'./{data_dir}'), transform)
    print("데이터 정보", dataset)

    meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in dataset]
    stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in dataset]

    meanR = np.mean([m[0] for m in meanRGB])
    meanG = np.mean([m[1] for m in meanRGB])
    meanB = np.mean([m[2] for m in meanRGB])

    stdR = np.mean([s[0] for s in stdRGB])
    stdG = np.mean([s[1] for s in stdRGB])
    stdB = np.mean([s[2] for s in stdRGB])
    print("평균",meanR, meanG, meanB)
    print("표준편차",stdR, stdG, stdB)

data_dir = "./data/train"
get_mean_std(data_dir)



데이터 정보 Dataset ImageFolder
    Number of datapoints: 12800
    Root location: ././data/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
평균 0.451251 0.4218885 0.39344847
표준편차 0.22973368 0.22437534 0.21690977


In [36]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset
import numpy as np
import cv2
import os
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [37]:
import torch
import random
import torch.backends.cudnn as cudnn

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(SEED)

# CustomData set

In [6]:
class CustomDataset(Dataset):

    def __init__(self, dfpath, transform=None):
        self.df = pd.read_csv(dfpath, usecols=["path", "label"])
        self.image_names = self.df["path"]
        self.label = self.df["label"]
        self.transform = transform

    def __getitem__(self, index):
        impath = os.path.join(self.image_names[index])
        image = cv2.imread(impath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        target = self.label[index]
        return image, target

    def __len__(self):
        return len(self.image_names)

In [5]:
# 데이터 증강을 위한 변환 기법 설정
train_transform = A.Compose([
    # efficient Net의 default size로 이미지 크기 조정
    A.Resize(224, 224),
    # 위에서 구한 train dataset에 대한 평균과 표준편차를 사용하여 정규화
    A.Normalize(
        mean=(0.451251, 0.4218885, 0.39344847),
        std=(0.22973368, 0.22437534, 0.21690977),
        max_pixel_value=225.0,
        always_apply=True,
        p=1.0
    ),
    A.Rotate(limit=45, p=0.3),  # 무작위로 이미지 회전 (90도까지)
    A.HorizontalFlip(p=0.2),  # 무작위로 이미지를 좌우로 뒤집음
    A.RandomBrightness(p=0.2),  # 무작위로 이미지의 밝기를 조정
    A.ColorJitter(brightness=0.2, contrast=0.2),
    # A.GaussianBlur(),
    # A.RandomContrast(limit=0.2, p=0.5),  # 무작위로 이미지의 대비를 조정
    # A.RandomGamma(p=0.5),  # 무작위로 이미지의 감마 값을 조정
    # A.Blur(p=0.2),  # 이미지를 블러링
    A.Cutout(num_holes=3, max_h_size=8, max_w_size=8, p=0.3),  # 이미지 내 임의의 영역을 잘라냄
    A.RandomCrop(height=224, width=224, p=0.2),  # 무작위로 이미지를 자르고 크기 조정
    ToTensorV2()  # 이미지를 텐서 형태로 변환
])


test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(
        mean=(0.451251, 0.4218885, 0.39344847),
        std=(0.22973368, 0.22437534, 0.21690977),
        max_pixel_value=225.0,
        always_apply=True,
        p=1.0
    ),
    ToTensorV2()
])

In [48]:
# dataset 생성
train_dataset = CustomDataset(dfpath="./train.csv", transform=train_transform)
val_dataset = CustomDataset(dfpath="./val.csv", transform=test_transform)

In [49]:
# train_dataloader
train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = 32,
    num_workers=0,
    shuffle=True
)

# val_dataloader
val_dataloader = DataLoader(
    dataset = val_dataset,
    batch_size = 64,
    num_workers=0,
    shuffle=False
)

# Define Model

In [51]:
class Efficientnet_b6(nn.Module):

    def __init__(self, num_classes=2):
        super(Efficientnet_b6, self).__init__()
        self.backbone = models.efficientnet_b6(pretrained=True)
        self.dropout = nn.Dropout(p=0.5)
        self.classifier = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        out = self.classifier(x)

        return out

model = Efficientnet_b6()

print(model)

Efficientnet_b6(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 56, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(56, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=56, bias=False)
              (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(56, 14, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(14, 56, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
             

# Model Compile

In [52]:
# Warmup Scheduler
class WarmupLR(optim.lr_scheduler.LambdaLR):

    def __init__(
        self,
        optimizer: optim.Optimizer,
        warmup_end_steps: int,
        last_epoch: int = -1,
    ):

        def wramup_fn(step: int):
            if step < warmup_end_steps:
                return float(step) / float(max(warmup_end_steps, 1))
            return 1.0

        super().__init__(optimizer, wramup_fn, last_epoch)

In [53]:
# define loss
loss_function = nn.CrossEntropyLoss()

# deifine model
model = Efficientnet_b6(num_classes=2)
model_name = type(model).__name__

gpu = 0
if gpu is not None:
    model.cuda(gpu)

# define optimizer
lr = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer_name = type(optimizer).__name__

# define scheduler
scheduler = WarmupLR(optimizer, 1500)
#scheduler = None
scheduler_name = type(scheduler).__name__ if scheduler is not None else "no"

max_epoch = 500

run_name = f"{model_name}4-{optimizer_name}_optim_{lr}_lr_with_{scheduler_name}_scheduler"
#run_name = f"{model_name}"
# define wandb
project_name = "fake-or-real"
run_tags = [project_name]
wandb.init(
    project=project_name,
     name=run_name,
     tags=run_tags,
     config={"lr": lr, "model_name": model_name, "optimizer_name": optimizer_name, "scheduler_name": scheduler_name},
     reinit=True,
 )
wandb.watch(model)

PROJECT_DIR = os.getcwd()
RUN_DIRNAME = "fake-or-real"

log_dir = os.path.join(PROJECT_DIR, "runs", RUN_DIRNAME, run_name)

# set save model path
log_model_path = os.path.join(log_dir, "models")
os.makedirs(log_model_path, exist_ok=True)\

log_interval = 1600

Acc/train,▁▂▃▄▆▇█
Acc/val,▁▅████████████
F1score/val,▁▅████████████
Learning Rate,▁██████
Loss/train,▁▆▅▆▇██
Loss/val,█▅▁▁▁▁▁▁▁▁▁▁▁▁
Acc/train,9.505
Acc/val,0.97993
F1score/val,0.9799
Learning Rate,0.0001
Loss/train,0.02899


# Define Early Stopping

In [54]:
# With some modifications, source is from https://github.com/Bjarten/early-stopping-pytorch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.ckpt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.ckpt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')

        filename = self.path.split('\\')[-1]
        save_dir = os.path.dirname(self.path)
        torch.save(model, os.path.join(save_dir, f"val_loss-{val_loss}-{filename}"))
        self.val_loss_min = val_loss

# 학습

In [55]:
# define EarlyStopping.
early_stopper = EarlyStopping(
    patience=5, verbose=True, path=os.path.join(log_model_path, "model.ckpt")
)

model = model.to("cuda")

# do train with validation.
train_step = 0
for epoch in range(1, max_epoch+1):

    # train step
    current_loss = 0
    current_corrects = 0
    model.train()

    for batch_idx, (images, labels) in enumerate(
         tqdm(train_dataloader, position=0, leave=False, desc="training",  mininterval=3)
    ):
        if gpu is not None:
            images = images.cuda(gpu)
            labels = labels.cuda(gpu)

        # Forward
        # get predictions
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        # get loss (Loss 계산)
        loss = loss_function(outputs, labels)

        # Backpropagation
        # optimizer 초기화 (zero화)
        optimizer.zero_grad()

        # Perform backward pass
        loss.backward()

        # Perform Optimization
        optimizer.step()

        # Perform LR scheduler Work
        if scheduler is not None:
            scheduler.step()

        current_loss += loss.item()
        current_corrects += torch.sum(preds == labels.data)

        if train_step % log_interval == 0:
            train_loss = current_loss / log_interval
            train_acc = current_corrects / log_interval

            print(
                f"{train_step}: train_loss: {train_loss}, train_acc: {train_acc}"
            )

            cur_lr = optimizer.param_groups[0]["lr"] if scheduler is None else scheduler.get_last_lr()[0]

            # wandb log
            wandb.log({
                 "Loss/train": train_loss,
                 "Acc/train": train_acc,
                 "Images/train": wandb.Image(images),
                 "Outputs/train": wandb.Histogram(outputs.detach().cpu().numpy()),
                 "Preds/train": wandb.Histogram(preds.detach().cpu().numpy()),
                 "Labels/train": wandb.Histogram(labels.data.detach().cpu().numpy()),
                 "Learning Rate": cur_lr,
             }, step=train_step)

            current_loss = 0
            current_corrects = 0

        train_step += 1

    # valid step
    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        model.eval()
        preds, true_labels = [], []

        for val_batch_idx, (val_images, val_labels) in enumerate(
            tqdm(val_dataloader, position=0, leave=False, desc="validation",  mininterval=3)
        ):
            if gpu is not None:
                val_images = val_images.cuda(gpu, non_blocking=True)
                val_labels = val_labels.cuda(gpu, non_blocking=True)
            # forward
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs, 1)
            preds += val_outputs.argmax(1).detach().cpu().numpy().tolist()
            true_labels += val_labels.detach().cpu().numpy().tolist()

            # loss & acc
            val_loss += loss_function(val_outputs, val_labels) / val_outputs.shape[0]
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0]


        # valid step logging
        val_epoch_loss = val_loss / len(val_dataloader)
        val_epoch_acc = val_corrects / len(val_dataloader)
        val_score = f1_score(true_labels, preds, average="macro")

        print(
            f"{epoch} epoch, {train_step} step: val_loss: {val_epoch_loss}, val_acc: {val_epoch_acc}, f1_score : {val_score}"
        )
        # wandb log
        wandb.log({
             "Loss/val": val_epoch_loss,
             "Acc/val": val_epoch_acc,
            "Images/val": wandb.Image(val_images),
             "Outputs/val": wandb.Histogram(val_outputs.detach().cpu().numpy()),
             "Preds/val": wandb.Histogram(val_preds.detach().cpu().numpy()),
             "Labels/val": wandb.Histogram(val_labels.data.detach().cpu().numpy()),
             "F1score/val" : val_score
         }, step=train_step)

        # check model early stopping point & save model if the model reached the best performance.
        early_stopper(val_epoch_loss, model)
        if early_stopper.early_stop:
            break

training:   0%|          | 0/759 [00:00<?, ?it/s]

0: train_loss: 0.00043733034282922745, train_acc: 0.011874999850988388


1 epoch, 759 step: val_loss: 0.007174901198595762, val_acc: 0.7871240973472595, f1_score : 0.7858528735309886
Validation loss decreased (inf --> 0.007175).  Saving model ...


2 epoch, 1518 step: val_loss: 0.003642955096438527, val_acc: 0.9052162170410156, f1_score : 0.9051368578927633
Validation loss decreased (0.007175 --> 0.003643).  Saving model ...


training:  10%|█         | 78/759 [00:59<08:43,  1.30it/s]

...

13 epoch, 9867 step: val_loss: 0.0009995308937504888, val_acc: 0.9802631735801697, f1_score : 0.980234375
EarlyStopping counter: 3 out of 5


14 epoch, 10626 step: val_loss: 0.0009691519662737846, val_acc: 0.9799342751502991, f1_score : 0.9799046252044846
EarlyStopping counter: 4 out of 5


training:  75%|███████▌  | 571/759 [07:04<02:19,  1.35it/s]

11200: train_loss: 0.03567542727847467, train_acc: 11.0337495803833


15 epoch, 11385 step: val_loss: 0.0010161985410377383, val_acc: 0.9800987243652344, f1_score : 0.9800704523906292
EarlyStopping counter: 5 out of 5


# Load Model

In [75]:
model = torch.load("./val_loss-0.0008872511680237949-model.ckpt")
model.eval()

Efficientnet_b6(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 56, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(56, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=56, bias=False)
              (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(56, 14, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(14, 56, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
             

In [76]:
class TestDataset(Dataset):

    def __init__(self, dfpath, transform=None):
        self.df = pd.read_csv(dfpath, usecols=["path"])
        self.image_names = self.df["path"]
        self.transform = transform

    def __getitem__(self, index):
        impath = os.path.join(self.image_names[index])
        image = cv2.imread(impath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image

    def __len__(self):
        return len(self.image_names)

In [77]:
test_dataset = TestDataset(dfpath="./test.csv", transform=test_transform)

In [85]:
test_dataloader = DataLoader(
    dataset = test_dataset,
    batch_size = 32,
    num_workers=0,
    shuffle=False
)

In [86]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inference

In [83]:
from tqdm import tqdm

In [87]:
# Make predictions
y_preds = []

for batch_index, (x) in enumerate(tqdm(test_dataloader)):
    x = x.to(device, dtype=torch.float)
    y_logits = model(x).cpu()
    y_pred = torch.argmax(y_logits, dim=1)
    y_logits = y_logits.detach().numpy()
    y_pred = y_pred.detach().numpy()
    for yp in y_pred:
        y_preds.append(yp)

100%|██████████| 1824/1824 [18:12<00:00,  1.67it/s]


In [88]:
sub = pd.read_csv("./data/test/sample_submission.csv")

sub

,ImageId,answer
0,test_00000.png,1
1,test_00001.png,1
2,test_00002.png,1
3,test_00003.png,1
4,test_00004.png,1
...,...,...
58343,test_58343.png,1
58344,test_58344.png,1
58345,test_58345.png,1
58346,test_58346.png,1


In [52]:
all_predictions

array([[0.34035355, 0.61360687],
       [0.9563197 , 0.03174191],
       [0.5097331 , 0.49900025],
       ...,
       [0.95273584, 0.04199295],
       [0.01875385, 0.9850363 ],
       [0.9665244 , 0.03056631]], dtype=float32)

In [89]:
sub["answer"].value_counts()

answer
1    58348
Name: count, dtype: int64

In [90]:
sub["answer"] = y_preds

In [91]:
sub["answer"].value_counts()

answer
0    42772
1    15576
Name: count, dtype: int64

In [99]:
sub.to_csv("effientb6.csv", index=False)